In [ ]:
# =========================================================
# 📦 IMPORTACIONES
# =========================================================
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# =========================================================
# ⚙️ CONFIGURACIÓN DE GPU
# =========================================================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        dev_names = [tf.config.experimental.get_device_details(g).get("device_name", str(g)) for g in gpus]
        print(f"✅ GPUs detectadas: {len(gpus)} -> {dev_names}")
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"   Logical GPUs: {len(logical_gpus)}")
    except RuntimeError as e:
        print("⚠️ Error al configurar GPUs:", e)
else:
    print("⚠️ No se detectó GPU, se usará CPU")

# --- Opcional: activar mixed precision ---
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
# (si usas esto, recuerda que la última capa Dense debe ser dtype='float32')

In [ ]:
# =========================================================
# 📂 CARGAR RUTAS
# =========================================================

# Rutas actualizadas para la nueva estructura
BASE_DIR = "images"
META_DIR = "meta/meta"

# Verificar que las rutas existen
if os.path.exists(BASE_DIR):
    print(f"Se encontró la ruta: {BASE_DIR} exitosamente!")
if os.path.exists(META_DIR):
    print(f"Se encontró la ruta: {META_DIR} exitosamente!")

In [ ]:
# =========================================================
# 📄 LECTURA DE SPLITS (train/test)
# =========================================================
def load_split(filename):
    path = os.path.join(META_DIR, filename).replace("\\", "/")
    try:
        with open(path, "r") as f:
            lines = f.read().splitlines()
        return lines
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo {path}")
        return []

train_files = load_split("train.txt")
test_files = load_split("test.txt")

print(f"📊 Train muestras: {len(train_files)}")
print(f"📊 Test muestras: {len(test_files)}")

if len(train_files) == 0 or len(test_files) == 0:
    print("❌ Error: No se pudieron cargar los archivos de splits")

In [ ]:
# =========================================================
# 📊 CREAR DATAFRAMES PARA TRAIN Y TEST
# =========================================================
def create_dataframe(file_list, base_dir):
    """Crear DataFrame con rutas de imágenes y etiquetas"""
    data = []
    for file_path in file_list:
        # Extraer clase del nombre del archivo (formato: clase/imagen.jpg)
        parts = file_path.split('/')
        if len(parts) >= 2:
            class_name = parts[0]
            full_path = os.path.join(base_dir, file_path).replace("\\", "/") + ".jpg"
            data.append({
                'filename': full_path,
                'class': class_name
            })
    
    return pd.DataFrame(data)

# Crear DataFrames
train_df = create_dataframe(train_files, BASE_DIR)
test_df = create_dataframe(test_files, BASE_DIR)

print(f"📊 Train DataFrame shape: {train_df.shape}")
print(f"📊 Test DataFrame shape: {test_df.shape}")
print(f"📊 Número de clases: {train_df['class'].nunique()}")
print(f"📊 Clases encontradas: {sorted(train_df['class'].unique())}")

# Verificar algunas imágenes
print("\n🔍 Verificando existencia de algunas imágenes:")
sample_files = train_df['filename'].head().tolist()
for file_path in sample_files:
    exists = os.path.exists(file_path)
    print(f"{'✅' if exists else '❌'} {file_path}")

In [ ]:
# =========================================================
# 🏗️ CONFIGURACIÓN DE GENERADORES DE DATOS
# =========================================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation para entrenamiento
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

# Solo preprocesssing para test
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Crear generadores
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Calcular steps
train_steps = train_generator.samples // BATCH_SIZE
val_steps = val_generator.samples // BATCH_SIZE
test_steps = test_generator.samples // BATCH_SIZE

print("✅ Generadores creados:")
print(f"📊 Train steps: {train_steps}")
print(f"📊 Validation steps: {val_steps}")
print(f"📊 Test steps: {test_steps}")
print(f"📊 Total train images: {train_generator.samples}")
print(f"📊 Total validation images: {val_generator.samples}")
print(f"📊 Total test images: {test_generator.samples}")
print(f"📊 Número de clases: {len(train_generator.class_indices)}")

# Verificar un batch
try:
    sample_batch = next(train_generator)
    print(f"✅ Batch de prueba: {sample_batch[0].shape}, {sample_batch[1].shape}")
except Exception as e:
    print(f"❌ Error al obtener batch de prueba: {e}")

In [ ]:
try:
    sample_batch = next(val_generator)
    print(f"✅ Batch de prueba del val_generator: {sample_batch[0].shape}, {sample_batch[1].shape}")
except Exception as e:
    print(f"❌ Error al obtener batch de prueba del val_generator: {e}")

In [ ]:
# =========================================================
# 🏗️ CONSTRUCCIÓN DEL MODELO ResNet50
# =========================================================
NUM_CLASSES = len(train_generator.class_indices)

# Cargar ResNet50 pre-entrenado sin la capa top
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMG_SIZE, 3)
)

# Agregar capas personalizadas
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions', dtype='float32')(x)

# Crear el modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

print(f"✅ Modelo creado con {NUM_CLASSES} clases")
print(f"📊 Parámetros totales: {model.count_params():,}")

In [ ]:
# =========================================================
# 🎯 ENTRENAMIENTO POR FASES
# =========================================================

# ---- Fase 1: Entrenar solo el clasificador ----
print("\n" + "="*50)
print("🔒 FASE 1: Entrenando solo el clasificador")
print("="*50)

# Congelar ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Compilar para Fase 1
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
early_stop_fase1 = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

checkpoint_fase1 = ModelCheckpoint(
    "best_model_fase1.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Entrenar Fase 1
print("🚀 Entrenando Fase 1 (solo capas densas)...")
history_fase1 = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=10,
    callbacks=[early_stop_fase1, checkpoint_fase1],
    verbose=1
)

# ---- Fase 2: Fine-tuning ----
print("\n" + "="*50)
print("🔓 FASE 2: Fine-tuning (desbloqueando últimas capas)")
print("="*50)

# Desbloquear las últimas capas de ResNet50
for layer in base_model.layers[-50:]:  # últimas 50 capas
    layer.trainable = True

# Compilar para Fase 2 con learning rate más bajo
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks para Fase 2
early_stop_fase2 = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True,
    verbose=1
)

checkpoint_fase2 = ModelCheckpoint(
    "best_model_fase2.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Entrenar Fase 2
print("🚀 Entrenando Fase 2 (fine-tuning)...")
history_fase2 = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=15,
    callbacks=[early_stop_fase2, checkpoint_fase2],
    verbose=1
)

In [ ]:
# =========================================================
# 📊 EVALUACIÓN FINAL
# =========================================================
print("\n" + "="*50)
print("📊 EVALUACIÓN FINAL")
print("="*50)

# Evaluar en conjunto de test
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_steps,
    verbose=1
)

print(f"✅ Test Loss: {test_loss:.4f}")
print(f"✅ Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# =========================================================
# 💾 GUARDADO DEL MODELO FINAL
# =========================================================
model_path = "model_resnet50_final.h5"
model.save(model_path)
print(f"✅ Modelo final guardado en: {model_path}")

# Guardar también los pesos por separado
weights_path = "model_resnet50_weights.h5"
model.save_weights(weights_path)
print(f"✅ Pesos guardados en: {weights_path}")

In [ ]:
# =========================================================
# 📈 VISUALIZACIÓN DE HISTORIAL (OPCIONAL)
# =========================================================
def plot_training_history(history1, history2, title1="Fase 1", title2="Fase 2"):
    """Función para plotear el historial de entrenamiento"""
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Fase 1
    axes[0,0].plot(history1.history['accuracy'], label='Train Accuracy')
    axes[0,0].plot(history1.history['val_accuracy'], label='Val Accuracy')
    axes[0,0].set_title(f'{title1} - Accuracy')
    axes[0,0].legend()
    
    axes[0,1].plot(history1.history['loss'], label='Train Loss')
    axes[0,1].plot(history1.history['val_loss'], label='Val Loss')
    axes[0,1].set_title(f'{title1} - Loss')
    axes[0,1].legend()
    
    # Fase 2
    axes[1,0].plot(history2.history['accuracy'], label='Train Accuracy')
    axes[1,0].plot(history2.history['val_accuracy'], label='Val Accuracy')
    axes[1,0].set_title(f'{title2} - Accuracy')
    axes[1,0].legend()
    
    axes[1,1].plot(history2.history['loss'], label='Train Loss')
    axes[1,1].plot(history2.history['val_loss'], label='Val Loss')
    axes[1,1].set_title(f'{title2} - Loss')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

# Descomentar para visualizar
# plot_training_history(history_fase1, history_fase2)

print("\n🎉 ¡Entrenamiento completado!")

In [ ]:
import tensorflow as tf
import sys
import platform

print("=" * 50)
print("🔍 VERIFICACIÓN DE CONFIGURACIÓN GPU")
print("=" * 50)

# Información básica de TensorFlow y Python
print(f"📦 TensorFlow version: {tf.__version__}")
print(f"🐍 Python version: {sys.version}")
print(f"💻 Sistema: {platform.system()} {platform.release()}")

# Verificar GPUs físicas
physical_gpus = tf.config.list_physical_devices('GPU')
print(f"\n🎮 GPUs físicas detectadas: {len(physical_gpus)}")

if physical_gpus:
    for i, gpu in enumerate(physical_gpus):
        print(f"   GPU {i}: {gpu}")
        
        # Obtener detalles de la GPU
        try:
            details = tf.config.experimental.get_device_details(gpu)
            print(f"      Device Name: {details.get('device_name', 'N/A')}")
            print(f"      Compute Capability: {details.get('compute_capability', 'N/A')}")
        except Exception as e:
            print(f"      No se pudieron obtener detalles adicionales: {e}")
        
        # Configurar memory growth
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            print(f"      ✅ Memory growth habilitado")
        except Exception as e:
            print(f"      ❌ Error configurando memory growth: {e}")

# Verificar GPUs lógicas
logical_gpus = tf.config.list_logical_devices('GPU')
print(f"\n🧠 GPUs lógicas: {len(logical_gpus)}")
for i, gpu in enumerate(logical_gpus):
    print(f"   Logical GPU {i}: {gpu}")

# Test de operación en GPU
if physical_gpus:
    print(f"\n🧪 Probando operación en GPU...")
    try:
        with tf.device('/GPU:0'):
            # Test simple de multiplicación de matrices
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            c = tf.matmul(a, b)
            
        print(f"✅ Operación en GPU exitosa")
        print(f"   Input shape: {a.shape} x {b.shape}")
        print(f"   Output shape: {c.shape}")
        print(f"   Device usado: {c.device}")
        
    except Exception as e:
        print(f"❌ Error en operación GPU: {e}")
else:
    print(f"\n⚠️  No se detectaron GPUs - usando CPU")

# Información de CUDA y capacidades
print(f"\n🔧 INFORMACIÓN DE SOPORTE:")
print(f"   Built with CUDA: {tf.test.is_built_with_cuda()}")

# Test de disponibilidad de GPU (método más moderno)
gpu_available = len(tf.config.list_physical_devices('GPU')) > 0
print(f"   GPU disponible: {gpu_available}")

# Información adicional de build
try:
    build_info = tf.sysconfig.get_build_info()
    print(f"   CUDA version: {build_info.get('cuda_version', 'N/A')}")
    print(f"   cuDNN version: {build_info.get('cudnn_version', 'N/A')}")
except:
    print("   No se pudo obtener información de build")

# Información de memoria GPU
if physical_gpus:
    try:
        # Intentar obtener información de memoria
        gpu_details = tf.config.experimental.get_memory_info('GPU:0')
        current_mb = gpu_details['current'] / (1024**2)
        peak_mb = gpu_details['peak'] / (1024**2)
        print(f"\n💾 MEMORIA GPU:")
        print(f"   Memoria actual: {current_mb:.1f} MB")
        print(f"   Memoria pico: {peak_mb:.1f} MB")
    except Exception as e:
        print(f"\n💾 Información de memoria no disponible: {e}")
        
    # Test de creación de tensor grande para verificar memoria
    try:
        print(f"\n🧪 Test de memoria GPU...")
        with tf.device('/GPU:0'):
            # Crear tensor de prueba (aproximadamente 100MB)
            large_tensor = tf.random.normal((5000, 5000), dtype=tf.float32)
            result = tf.reduce_sum(large_tensor)
        print(f"✅ Test de memoria exitoso: suma = {result:.2f}")
        del large_tensor  # Liberar memoria
    except Exception as e:
        print(f"❌ Error en test de memoria: {e}")

print("\n" + "=" * 50)
print("🎉 Verificación completada")
print("=" * 50)